# Routing stack test

* Date: 2023-01-05
* Issue: <https://github.com/WasteLabs/streamlit_example_routing_app/issues/1>

## Scope

Test the following routing stack:

* <https://openrouteservice.org/dev/#/api-docs>: for the routing
* <https://github.com/gis-ops/routingpy>: to set it up
* <https://github.com/VROOM-Project/pyvroom>: for routing optimisation

## Imports and jupyter setup

In [2]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# GLOBAL VARS
from pathlib import Path
import sys
PROJECT_DIR = os.getcwd().rsplit('/', 1)[0]

if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

/var/folders/dy/_xqj_wpj3jl0lgk31f6ytns00000gp/T/ipykernel_24783/1677521232.py:20: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Test operouteservice

Linked with <https://github.com/WasteLabs/streamlit_example_routing_app/issues/2>, we get a local instance running at localhost <http://localhost:8080/>.
